In [ ]:
import torch
from torch_geometric.nn import GCNConv

from soccerai.data.dataset import WorldCup2022Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

index
gameId
gameEventId
possessionEventId
startTime
endTime


teniamo:
per il tempo
- frameTime (convertire in secondi)
- duration
- possessionEventType (OneHot)
- playerName va trasformato in un valore booleano che indica il giocatore che ha la palla ma di quel nodo (ovvero playerName == playerName_right)
- si tiene team per partizionare i giocatori in due squadre (OneHot) (perchè non ordinale? Trello)
- indicare anche il Team che ha la palla (booleano)
- x, y
- velocity
- direction
- playerId (oneHot)
- playerRole (oneHot)
- height
- weight
- age
- goals
- shots


togliamo:
- index
- gameId
- gameEventId
- possessionEventId
- startTime
- endTime
- gameEventType
- teamName va droppato per i motivi spiegati (da scrivere su Trello)
- videoUrl
- z si droppa
- tutte le righe con la palla si droppano
- label si droppa da X
- index_right
- gameId_right
- jerseyNum
- visibility
- homeTeamName
- awayTeamName
- playerName_right
- Full Name
- birthdate



vogliamo far sapere alla rete che squadra sta giocando? e forza una sorta di conoscenza della squadra all'intenro della rete stessa?
SIAMO DUBBIOSI

In [ ]:
dataset = WorldCup2022Dataset(
    "/home/aarcara/soccerai/soccerai/data/resources", "fully_connected"
)
dataset.process()

In [ ]:
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import global_add_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)

        x = global_add_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x


model = GCN(hidden_channels=64)
model.to(device)
data = dataset[0].to(device)
model(data)

In [ ]:
import os

import torch
import torch.nn as nn
from torch_geometric.loader import DataLoader
from tqdm.notebook import tqdm

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

loader = DataLoader(
    dataset[:-2],
    128,
    num_workers=os.cpu_count() - 1,
    shuffle=True,
    pin_memory=True,
    persistent_workers=True,
)

In [ ]:
dataset[0].x

In [ ]:
!pip install tabulate

In [ ]:
from torch_geometric.nn import summary

print(summary(model, dataset[0].to(device)))

In [ ]:
for param in model.parameters():
    print(param.size())

In [ ]:
model.train()
for epoch in tqdm(range(100), desc="Epoch"):
    for batch in tqdm(loader, desc="Batch", leave=False):
        optimizer.zero_grad()

        batch.to(device, non_blocking=True)

        out = model(batch)

        loss = criterion(out, batch.y.float().unsqueeze(1))

        loss.backward()
        optimizer.step()

In [ ]:
dataset[-7].y

In [ ]:
F.sigmoid(model(dataset[-7].to(device)))